# MP2 相关基础

尽管 MP2 的了解对非自洽 DFT 的问题没有帮助，但我们可以将这一节作为使用 SCF 结果的一个范例．

In [ ]:
import numpy as np
from pyscf import scf, gto, lib, mp
from functools import partial

import sys
sys.path.append('../../src')
from utilities import val_from_fchk

np.einsum = partial(np.einsum, optimize=["greedy", 1024 ** 3 * 2 / 8])
np.set_printoptions(5, linewidth=150, suppress=True)

In [ ]:
mol = gto.Mole()
mol.atom = """
O  0.0  0.0  0.0
O  0.0  0.0  1.5
H  1.5  0.0  0.0
H  0.0  0.7  1.5
"""
mol.basis = "6-31G"
mol.verbose = 0
mol.build()

In [ ]:
scf_eng = scf.RHF(mol)
scf_eng.conv_tol = 1e-12
scf_eng.conv_tol_grad = 1e-10
scf_eng.kernel()

## PySCF MP2 计算

PySCF 的 MP2 计算可以通过下述代码实现：

In [ ]:
mp2_eng = mp.MP2(scf_eng)
mp2_eng.kernel()[0]

上述的输出是体系的相关矫正能 $E_\mathrm{MP2, c}$．

我们可以将上述计算结果与 [Gaussian 结果](include/basic_mp2/mp2_energy.gjf) 进行比对：

In [ ]:
np.allclose(
    val_from_fchk("MP2 Energy", "include/basic_mp2/mp2_energy.fchk"),
    mp2_eng.e_tot
)

## 小型 MP2 程序

在我们继续 MP2 程序之前，我们需要定义一些 SCF 过程中所经常用到的变量．

### 分子相关量定义

* `nmo` 分子电子数

* `nao` 原子轨道数

* `natm` 原子数

* `nocc` 占据轨道数

* `nvir` 未占轨道数

* `so` 占据轨道分割

* `sv` 未占轨道分割

* `sa` 全轨道分割

In [ ]:
nmo = nao = mol.nao
natm = mol.natm
nocc = mol.nelec[0]
nvir = nmo - nocc
so = slice(0, nocc)
sv = slice(nocc, nmo)
sa = slice(0, nmo)

### 自洽场计算相关量

* `C` 系数矩阵 $C_{\mu p}$

* `e` 轨道能量 $\varepsilon_p$

* `Co` 占据轨道系数矩阵 $C_{\mu i}$

* `Cv` 未占轨道系数矩阵 $C_{\mu a}$

* `eo` 占据轨道能量 $\varepsilon_i$

* `ev` 未占轨道能量 $\varepsilon_a$

* `D` 密度矩阵 $D_{\mu \nu}$

* `F_0_ao` AO 基组 Fock 矩阵 $F_{\mu \nu}$

* `F_0_mo` MO 基组 Fock 矩阵 $F_{pq}$ (为对角阵)

* `H_0_ao` AO 基组 Hamiltonian Core 矩阵 $h_{\mu \nu}$

* `H_0_mo` MO 基组 Hamiltonian Core 矩阵 $h_{pq}$

* `eri0_ao` AO 基组双电子互斥积分 $(\mu \nu | \kappa \lambda)$

* `eri0_mo` MO 基组双电子互斥积分 $(pq | rs)$

* `mo_occ` 轨道占据数

In [ ]:
C       = scf_eng.mo_coeff
e       = scf_eng.mo_energy
Co      = C[:, so]
Cv      = C[:, sv]
eo      = e[so]
ev      = e[sv]
D       = 2 * Co @ Co.T
F_0_ao  = scf_eng.get_fock()
F_0_mo  = C @ F_0_ao @ C.T
H_0_ao  = scf_eng.get_hcore()
H_0_mo  = C @ H_0_ao @ C.T
eri0_ao = mol.intor("int2e")
eri0_mo = np.einsum("uvkl, up, vq, kr, ls -> pqrs", eri0_ao, C, C, C, C)
mo_occ  = scf_eng.mo_occ

### MP2 计算

事实上刚刚我们已经把 MP2 中计算量最大的部分，即 MO 基组的原子轨道已经生成出来了．在 RHF 下，MP2 计算表现为 (Szabo, (6.74))

$$
E_\mathrm{MP2, c} = \frac{(ia|jb) \big( 2 (ia|jb) - (ib|ja) \big)}{\varepsilon_i - \varepsilon_a + \varepsilon_j - \varepsilon_b}
$$

因此，剩下来需要我们做的事仅仅是

* 定义关于 $i, a, j, b$ 的张量 `e_ovov` $\varepsilon_{ij}^{ab} = \varepsilon_i - \varepsilon_a + \varepsilon_j - \varepsilon_b$ 

* 为了程序书写方便而新建一个 ndarray 的视图 (View，即一个 ndarray 的浅复制) `eri0_ovov` $(ia|jb)$

In [ ]:
e_ovov = eo[:, None, None, None] - ev[None, :, None, None] + eo[None, None, :, None] - ev[None, None, None, :]
eri0_ovov = eri0_mo[so, sv, so, sv]
E_MP2c = (eri0_ovov * (2 * eri0_ovov - eri0_ovov.swapaxes(1, 3)) / e_ovov).sum()

我们能验证生成出来的 MP2 相关能的结果：

In [ ]:
np.allclose(
    mp2_eng.emp2,
    E_MP2c
)